# NoteBook To Check Point or Polygon Inside Polygon

The objective of this notebook is to search a set of points or polygons for spatial overlap with a largerset

By Cascsde Tuholske 2019-01-17

#### Notes
1. Points should be dialated to circles to prevent one falling just outside the larger polygon set (see Lusaka)

### Depedencies

In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import mapping
from shapely.geometry import Polygon
from shapely.geometry import shape
import ast
from shapely.geometry import mapping
import rasterio
import copy

### Functions

In [3]:
def load_points (file):
    """ This function loads a csv 
    of points and turns it into shapely points"""
    df = pd.read_csv(file)

    # creating a geometry column 
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

    # Coordinate reference system : WGS84
    crs = {'init': 'epsg:4326'}

    # Creating a Geographic data frame 
    point_gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
    
    return point_gdf

In [4]:
def point_buffer(gpd_df, raduis):
    "Function to make a shapely polygon buffer around a point"
   
    # AGU 2018-12-04 - radius set to ~250m at the equator 
    #radius = 250*1/(111*1000)

    # new_gpd_df = gpd.GeoDataFrame()
    gpd_df.rename(columns={'geometry':'old_geom'}, inplace=True)
    
    arr = []
    
    for point in gpd_df['old_geom']:
        buffer = point.buffer(radius)
        arr.append((buffer))
    
    gpd_df['geometry'] = arr
    
    return gpd_df

In [5]:
def poly_point (poly, point):
    """
    This function will check if points are inside polygons if given two gpd dataframes with points and polygons
    Returns point ids, point geometry, polygon index # and polygon geometry
    """
    
    out_arr = [] #return an array <<< ---------------- ASK RYAN IF BETTER DO USE DICT 
    
    for index_point, row_point in point.iterrows():
        for index_poly, row_poly in poly.iterrows():
            if row_point['geometry'].within(row_poly['geometry']):
                point_id = row_point['id']
                point_geom = mapping(row_point['geometry']) # makes a dict w/ keys : type and cood
                poly_id = index_poly
                poly_geom = mapping(row_poly['geometry']) # makes a dict w/ keys : type and cood
                
                out_arr.append((point_id, 
                                point_geom, 
                                poly_id, 
                                poly_geom))

    return out_arr

# Note 2018-11-30 update arr to gpd_df 

In [6]:
def poly_overlap (point_buffer, poly_raster):
    """
    This function will check if point buffers intersect with polygons 
    if given two gpd dataframes with point buffers and polygons
    Returns point ids, point geometry, polygon index #, and polygon geometry in a geopandas DF.
    It goes faster if smaller list goes first
    
    Change TOWN & CITY ARRA
    """
    
    # make arrays to fill
    country_arr = []
    city_arr = []
    osm_type_arr = []
    lat_arr = []
    lon_arr = []
    osm_id_arr = [] 
    FID_arr = [] 
    poly_geom_arr = []
    #point_geom_arr = []
    
    for index_point_buffer, row_point_buffer in point_buffer.iterrows():
        for index_poly_raster, row_poly_raster in poly_raster.iterrows():
            if row_point_buffer['geometry'].intersects(row_poly_raster['geometry']):
                country = row_point_buffer['country']
                city = row_point_buffer['town'] # Need to change town vs city
                osm_type = row_point_buffer['osm_type']
                lat = row_point_buffer['lat']
                lon = row_point_buffer['lon']
                osm_id = row_point_buffer['osm_id']
                poly_id = row_poly_raster['FID']
                poly_geom = shape(mapping(row_poly_raster['geometry'])) # make polygon
                #point_geom = row_point_buffer['old_geom']

                country_arr.append((country))
                city_arr.append((city))
                osm_type_arr.append((osm_type))
                lat_arr.append((lat))
                lon_arr.append((lon))
                osm_id_arr.append((osm_id))
                FID_arr.append((poly_id))
                poly_geom_arr.append((poly_geom))
                #point_geom_arr.append((point_geom))
    
    # put results into a geopandas df
    new_gpd_df = gpd.GeoDataFrame()
    new_gpd_df['osm_id'] = osm_id_arr
    new_gpd_df['FID'] = FID_arr
    new_gpd_df['geometry'] = poly_geom_arr
    new_gpd_df['country'] = country_arr           
    new_gpd_df['city'] = city_arr
    new_gpd_df['osm_type'] = osm_type_arr
    new_gpd_df['lat'] = lat_arr
    new_gpd_df['lon'] = lon_arr
    #new_gpd_df['point_geom'] = point_geom_arr
    
    
    return new_gpd_df

# Note 2018-11-30 update arr to gpd_df 

# OSM Merge Cities and Towns

In [7]:
temp_data = '../../temp_data/'
towns = '20190221_osm_S_Sudan_towns.csv'
cities = '20190221_osm_S_Sudan_cities.csv'

towns_gdf = load_points(temp_data+towns)
cities_gdf = load_points(temp_data+cities)

towns_gdf['osm_type'] = 'town'
cities_gdf['osm_type'] = 'city'

print(len(towns_gdf))
print(len(cities_gdf))

osm_points = pd.concat([towns_gdf, cities_gdf], ignore_index=True, sort = False)
print(len(osm_points))

193
18
211


In [8]:
osm_points.tail()

,country,osm_id,town,lat,lon,geometry,osm_type
206,South Sudan,435669247,Yambio,4.568072,28.397670,POINT (28.39767 4.5680716),city
207,South Sudan,2096234512,Nimule,3.596935,32.060142,POINT (32.0601419 3.5969348),city
208,South Sudan,2613802518,Bentiu,9.255848,29.803550,POINT (29.8035497 9.2558478),city
209,South Sudan,2616003624,Renk,11.748914,32.816359,POINT (32.81635900000001 11.748914),city
210,South Sudan,3275722461,Rumbek,6.806390,29.676749,POINT (29.6767488 6.8063901),city


In [9]:
# AGU 2018-12-04 - radius set to ~250m at the equator 

radius = 250*1/(111*1000)
radius

0.0022522522522522522

In [10]:
osm_buffer_gdf = point_buffer(osm_points, radius)

In [11]:
osm_buffer_gdf.head(6)

,country,osm_id,town,lat,lon,old_geom,osm_type,geometry
0,South Sudan,83037194,Gogrial,8.531514,28.099653,POINT (28.0996528 8.531514400000001),town,"POLYGON ((28.10190505225225 8.531514400000001,..."
1,South Sudan,204367085,Kajo Keji,3.886761,31.668720,POINT (31.66872 3.886761),town,"POLYGON ((31.67097225225225 3.886761, 31.67096..."
2,South Sudan,205418847,Kapoeta,4.771099,33.588230,POINT (33.58823 4.771099),town,"POLYGON ((33.59048225225225 4.771099, 33.59047..."
3,South Sudan,205467269,Lainya,4.343070,31.072689,POINT (31.07268880000001 4.343069799999999),town,"POLYGON ((31.07494105225226 4.343069799999999,..."
4,South Sudan,205495747,Mundri,5.346478,30.322191,POINT (30.3221909 5.346478),town,"POLYGON ((30.32444315225225 5.346478, 30.32443..."
5,South Sudan,205496460,Mongalla,5.199872,31.769913,POINT (31.7699131 5.1998722),town,"POLYGON ((31.77216535225225 5.1998722, 31.7721..."


In [12]:
osm_buffer_gdf = osm_buffer_gdf.drop(['old_geom'], axis=1)
osm_buffer_gdf.head(6)

,country,osm_id,town,lat,lon,osm_type,geometry
0,South Sudan,83037194,Gogrial,8.531514,28.099653,town,"POLYGON ((28.10190505225225 8.531514400000001,..."
1,South Sudan,204367085,Kajo Keji,3.886761,31.668720,town,"POLYGON ((31.67097225225225 3.886761, 31.67096..."
2,South Sudan,205418847,Kapoeta,4.771099,33.588230,town,"POLYGON ((33.59048225225225 4.771099, 33.59047..."
3,South Sudan,205467269,Lainya,4.343070,31.072689,town,"POLYGON ((31.07494105225226 4.343069799999999,..."
4,South Sudan,205495747,Mundri,5.346478,30.322191,town,"POLYGON ((30.32444315225225 5.346478, 30.32443..."
5,South Sudan,205496460,Mongalla,5.199872,31.769913,town,"POLYGON ((31.77216535225225 5.1998722, 31.7721..."


### Analysis

In [13]:
# Load Polygons

polygons = 'GHS2000_1500c300_S_Sudan'

raster_polygons = gpd.read_file(temp_data+polygons+'.shp')
print(len(raster_polygons))

3002


In [14]:
raster_polygons.head(6)

,DN,geometry
0,1,"POLYGON ((25.35376583522545 13.66058518329869,..."
1,1,"POLYGON ((25.938444851536 13.59607069620633, 2..."
2,1,"POLYGON ((23.35278091580381 13.48979018016193,..."
3,1,"POLYGON ((23.70808668642125 13.48979018016193,..."
4,1,"POLYGON ((27.15905433966888 13.48161550633154,..."
5,1,"POLYGON ((23.30103812862056 13.47344093936391,..."


### Change CRS if needed

In [ ]:
raster_polygons.crs

In [ ]:
osm_buffer_gdf.crs

In [ ]:
# For GHS CRS

# crs = {'ellps': 'WGS84',
#  'lon_0': 0,
#  'no_defs': True,
#  'proj': 'moll',
#  'units': 'm',
#  'x_0': 0,
#  'y_0': 0}

In [ ]:
# raster_polygons.crs = crs

In [ ]:
# raster_polygons_crs = raster_polygons.to_crs(osm_buffer_gdf.crs)

In [21]:
# For GHS make FID and reset crs
FID = list(range(len(raster_polygons)))
raster_polygons['DN'] = FID
raster_polygons.columns.values[0] = "FID"

In [22]:
raster_polygons.head()

,FID,geometry,DN
0,0,"POLYGON ((25.35376583522545 13.66058518329869,...",0
1,1,"POLYGON ((25.938444851536 13.59607069620633, 2...",1
2,2,"POLYGON ((23.35278091580381 13.48979018016193,...",2
3,3,"POLYGON ((23.70808668642125 13.48979018016193,...",3
4,4,"POLYGON ((27.15905433966888 13.48161550633154,...",4


### Current Start 
1. Data: 2019-01-18 @ 6:22 pm
2. Raster Poly File: GHS_POP_GPW42000_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300.shp
3. OSM Town File: 20190114_osm_africa_towns.csv
4. Clip: [25000:35000]
5. outfile - GHS_POP_GPW42000_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300_subB1.shp

In [ ]:
# run poly_overlap

import time
checkpoint = time.time()

poly_overlap_out = poly_overlap(osm_buffer_gdf, raster_polygons)

print("elapsed time is: {}s".format(time.time()-checkpoint))
print(len(poly_overlap_out))

In [24]:
print(len(poly_overlap_out))

59


In [25]:
poly_overlap_out.tail(20)

,osm_id,FID,geometry,country,city,osm_type,lat,lon
39,3377513891,1271,"POLYGON ((31.88103310219279 7.911538930012074,...",South Sudan,Yuai,town,7.907144,31.887996
40,3377738680,1232,"POLYGON ((32.23462249124741 8.163206710726651,...",South Sudan,Walgak,town,8.159517,32.239775
41,3990702540,393,"POLYGON ((32.09557058170548 11.96291035213321,...",South Sudan,El Seriha,town,11.945706,32.106314
42,4047419041,811,"POLYGON ((31.39959151883566 9.292426580646307,...",South Sudan,Adiang,town,9.291506,31.408826
43,4047419111,798,"POLYGON ((31.40229861064124 9.341200039101437,...",South Sudan,Atar,town,9.337460,31.407884
44,4049845245,1529,"POLYGON ((31.65361344922741 6.345951655297514,...",South Sudan,Werngueny,town,6.340074,31.655953
45,97922073,1655,"POLYGON ((31.58475415692155 4.887467952694959,...",South Sudan,Juba,city,4.847202,31.595166
46,210414793,760,"POLYGON ((31.63561150214601 9.552582077458265,...",South Sudan,Malakal,city,9.541061,31.650426
47,227880249,1227,"POLYGON ((32.14546819666891 8.187564848663826,...",South Sudan,Waat,city,8.178370,32.151843
48,228735754,1701,"POLYGON ((30.66434022688865 4.102034439893941,...",South Sudan,Yei,city,4.092673,30.677393


In [26]:
poly_overlap_out.to_file(temp_data+polygons+'_polyoverlap.shp', driver='ESRI Shapefile')